# 2 Assignment: MNIST math
We are revisiting MNIST handwritten digits, but with a twist: We are not classifying handwritten digits, but rather asking our network to do some simple math on them: In subtask (1) we want to know whether a + b ≥ 5, in subtask (2) we want to predict y = a − b, where a and b are respective MNIST digits!
    In the following we ask you to write code (specifically your model and training data preparation) in a parameterized way, which allows you to reuse nearly all of the code for both subtasks, to minimize the effort required for this homework! We will try and give some hints on how to easily achieve this in the following. If there are any open questions, make sure to ask away in the QnA
sessions.

## 2.1 Preparing MNIST math dataset
Remember from last week:he MNIST dataset consists of seventy thousand labelled images, each depicting a single handwritten digit. This may sound like alot of data, but as you’ll see for yourselves in a bit, the images are rather small.
    The MNIST dataset is included in TensorFlow, so you getting access to it is actually pretty easy. You can load it directly into your code like this:

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import datetime
import tqdm

# Magic line only needed in jupyter notebooks!
%load_ext tensorboard

# Load mnist from tensorflow_datasets
train_ds, test_ds = tfds.load('mnist', split=['train', 'test'], as_supervised=True)

## 2.2 Two MNIST math datasets
Remember that for the following you want to create a preprocessing function.
As subtask (1) an substask (2) require very similar inputs, we will create a parameterized preprocessing function - mapping a dataset and the ’subtraction’ or ’larger than five’ condition onto respective datasets. We will generally have three different steps to our preprocssing - general MNIST preprocessing (datatype, normalization, etc.), pairing data tuples onto respective parameterized targets, and finally batching and prefetching.
    As a first step, for general preprocessing steps we follow last weeks ideas closely. You may follow the example code from the lecture (i.e. L03) again, but to summarize the steps:
    The MNIST handwritten digits images come in uint8 datatype. This refers to unsigned 8-bit integers (think numbers 0-255). As the network requires float values (think continuous variables) as input rather than integers (whole numbers), we need to change the datatype: (map in combination with lambda expressions can be really useful here). In your first lambda mapping you want to change the datatype from uint8 to tf.float values. To feed your network the 28x28 images also need to be flattened. Check out the reshape function, and if you want to minimize your work, try and understand how it interacts with size elements set to the value -1 (infering the remainder shape). In order to improve the performance you should also normalize your image values. Generally this means bringing the input close to the standart normal (gaussian) distribution with µ = 0 and σ = 1, however we can make a quick approximation as that: Knowing the inputs are in the 0-255 interval, we can simply divide all numbers by 128 (bringing them between 0-2), and finally subtracting one (bringing them into -1 to 1 range).
    Second Step is figuring out the parameterized targets: Think about the following: What kind of supervised learning task and respective loss-function and target type are required for our two subtasks respectively? It makes sense to work yourself through this backwards: First understand what loss function is required for each. Then check the respective TensorFlow documentation to find out how your targets should look. Once you know what target data-representation is needed for each, we can move to the second step: Each datapoint fed into the network requires two MNIST images as input, with one target being created from their combination. We will discuss many approaches to combining dataset elements - but the dataset zip functionality 5 probably is the most direct solution (as alternatives you might check out the respective window, scan or even
batch functions!). After pairing two dataset elements, you can now make sure each dataset element gets two input mnist images, but you still have to calculate the respective target as the third element.
    Before finalizing the dataset in the third step we recommend checking your dataset at this point - you expect each element to be a triple (tuple with three elements), the first two of which are images, the last one the respective target. Finally we shuffle and (mini-)batch the dataset, and can use the apply method to create datasets for both the a + b ≥ 5 and a − b = y problem!

In [2]:
# Create our dataset
def prepare_math_minst_data(math_mnist, batch_size, subtask):
    
    # math_mnist = math_mnist.map(lambda img_x, target: (tf.cast(img_x, tf.float32) / 128. - 1, target))
    # flatten the images into vector
    math_mnist = math_mnist.map(lambda img_x, target: (tf.reshape(img_x, (-1,)), target))
    # Convert data from uint8 to float32
    math_mnist = math_mnist.map(lambda img_x, target: (tf.cast(img_x, tf.float32), target))
    # Sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
    math_mnist = math_mnist.map(lambda img_x, target: ((img_x/128.)-1., target))
    # Shuffle data
    # we want to have two mnist images in each example
    # this leads to a single example being ((img_x1,img_y1),(img_x2,img_y2))
    zipped_dataset = tf.data.Dataset.zip((math_mnist.shuffle(2000),
                                        math_mnist.shuffle(2000)))
    # Subtask 1: a + b ≥ 5
    if subtask == 1:
        # Create targets
        zipped_dataset = zipped_dataset.map(lambda img_x1, img_x2: (img_x1[0], img_x2[0],
                                        tf.cast((img_x1[1] + img_x2[1] >= 5), tf.int32)))
    # Subtask 2: y == a - b 
    elif subtask == 2:
        # Create targets
        zipped_dataset = zipped_dataset.map(lambda img_x1, img_x2: (img_x1[0], img_x2[0],
                                        tf.cast((img_x1[1] - img_x2[1]), tf.int32)))
        zipped_dataset = zipped_dataset.map(lambda img_x1, img_x2, target: (img_x1, img_x2, 
                                        tf.one_hot(target, depth=19)))
    # Cache this progress in memory
    zipped_dataset.cache()
    # Shuffle data
    zipped_dataset = zipped_dataset.shuffle(2000)
    # Batch data
    zipped_dataset = zipped_dataset.batch(batch_size)
    # Prefetch data
    zipped_dataset = zipped_dataset.prefetch(tf.data.AUTOTUNE)
    # Return preprocessed dataset
    return zipped_dataset

# 3 Building shared weight models
In this step we have to build a network, that can solve the math task: This requires a network that takes two inputs (i.e. two input images), and outputs, i.e. predicts the math result. Technically you could just combine (e.g. concatenate) the two input images into one vector, however this would be rather inefficient. Instead we want you to learn about weight-sharing in a hands-on way: One intuitive way to solve this task would be to feed both inputs seperatedly into a layer, then combine (e.g. concatenate) the results from this layer. But both of these input layers would have to solve essentially the same problem for their respective input images: Extract the information, what digit is depicted! As both are basically tasked with the same problem, and have the same input and output shapes, we can improve our network by using the same layer for both inputs.  Finally you have to make sure to parameterize your model to use the correct activation functions for subtasks (1) and subtask (2) respectively.

In [3]:
class MyNNmodel(tf.keras.Model):
    def __init__(self, optimizer, subtask):
        # Inherit functionality from parent class
        super(MyNNmodel, self).__init__()
        # self.flatten = tf.keras.layers.Flatten()
        # Optimizer
        self.optimizer = optimizer
        self.subtask = subtask
        # layers to encode the images (both layers used for both images)
        self.dense1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        
        if self.subtask == 1:
            # loss function
            self.loss_function = tf.keras.losses.BinaryCrossentropy()
            # Metrix
            self.metrics_list = [
                tf.keras.metrics.BinaryAccuracy(name='accuracy'), 
                tf.keras.metrics.Mean(name='loss')
                ]
            self.out_layer = tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)

        elif self.subtask == 2:
            # loss function
            self.loss_function = tf.keras.losses.CategoricalCrossentropy()
            # Metrix
            self.metrics_list = [
                tf.keras.metrics.CategoricalAccuracy(name='accuracy'), 
                tf.keras.metrics.Mean(name='loss')
                ]
            self.out_layer = tf.keras.layers.Dense(19,activation=tf.nn.softmax)
            
    # Call method(forward computation)
    @tf.function
    def __call__(self, images, training=False):
        img1, img2 = images
        
        # img1_x = self.flatten(img1)
        # img2_x = self.flatten(img2)
        
        img1_x = self.dense1(img1)
        img1_x = self.dense2(img1_x)
        
        img2_x = self.dense1(img2)
        img2_x = self.dense2(img2_x)
        
        combined_x = tf.concat([img1_x, img2_x ], axis=1)
        combined_x = self.dense3(combined_x)
        return self.out_layer(combined_x)


    # Metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model

    # Reset all metrics objects
    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()

    # Train step method
    @tf.function
    def train_step(self, data):
        img1, img2, label = data

        with tf.GradientTape() as tape:
            output = self((img1, img2), training=True)
            loss = self.loss_function(label, output)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # Update the state of the metrics according to loss
        self.metrics[0].update_state(label, output)
        self.metrics[1].update_state(loss)

        # Return a dictionary with metric names as keys and metric results as values
        return {m.name: m.result() for m in self.metrics}

    # Test step method
    @tf.function
    def test_step(self, data):
        img1, img2, label = data

        # The same as train step(without parameter updates)
        output = self((img1, img2), training=True)
        loss = self.loss_function(label, output)

        # Update the state of the metrics according to loss
        self.metrics[0].update_state(label, output)
        self.metrics[1].update_state(loss)

        # Return a dictionary with metric names as keys and metric results as values
        return {m.name: m.result() for m in self.metrics}

# 4 Training the networks
Create a training function (with an internal training loop function), which is able to run either of the two subtasks. The function takes two inputs: The subtask to solve and the optimizer to use for it, both specified via inputs: Write the
function, such that it creates respective models and datasets based on the parameterized preprocessing function you have written before. Choose the correct
loss-function for the task and run training.

In [4]:
def create_summary_writers(config_name):
    # Define where to save the logs
    # along with this, you may want to save a config file with the same name so you know what the hyperparameters were used
    # Alternatively make a copy of the code that is used for later reference
    current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    train_log_path = f"logs/{config_name}/{current_time}/train"
    test_log_path = f"logs/{config_name}/{current_time}/train"

    # Log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)
    # log writer for validation metrics
    test_summary_writer = tf.summary.create_file_writer(test_log_path)
    
    return train_summary_writer, test_summary_writer

In [5]:
def train_loop(model, train_ds, test_ds, start_epoch, 
                epochs, train_summary_writer, 
                test_summary_writer, save_path):

    #1. Iterate over epochs
    for epoch in range(start_epoch, epochs):
        # 2. Train steps on all batches in the training data
        for data in tqdm.tqdm(train_ds, position=0, leave=True):
            metrics = model.train_step(data)
        
        # Print the Epoch number
        print(f'Epoch: {epoch}')
        
        # 3.log and print training metrics 
        with train_summary_writer.as_default():
            # for scalar metrics:
            for metric in model.metrics:
                tf.summary.scalar(f'{metric.name}', metric.result(), step=epoch)
            # Alternatively, log metrics individually (allows for non-scalar metrics such as tf.keras.metrics.MeanTensor)
            # e.g. tf.summary.image(name="mean_activation_layer3", data = metrics["mean_activation_layer3"],step=e)
        
        # Print the metrics
        print([f'{key}: {value.numpy()}' for (key, value) in metrics.items()])

        # 4. Reset metric objects
        model.reset_metrics()

        # 5. Evaluate on validation data
        for data in test_ds:
            metrics = model.test_step(data)

        # 6. Log validationmetric
        with test_summary_writer.as_default():
            # for scalar metrics:
            for metric in model.metrics:
                tf.summary.scalar(f'{metric.name}', metric.result(), step=epoch)
            # Alternatively, log metrics individually (allows for non-scalar metrics such as tf.keras.metrics.MeanTensor)

        print([f'test_{key}: {value.numpy()}' for (key, value) in metrics.items()])
        # 7. Reset metric objects
        model.reset_metrics()

    # 8. Save the model weghts if save_path is given
    if save_path:
        model.save_weights(save_path)

# 5  Experiments
Run training with a classic SGD optimizer (without momentum) and an Adam Optimizer. For an outstanding submission, you are required to also add SGD with Momentum, RMSProp and AdaGrad optimizers and compare the training results by plotting them cleanly side-by-side.

In [6]:
def train(optimizer, save_path, batch_size, subtask):
    
    train_summary_writer, test_summary_writer = create_summary_writers(config_name='RUN1')

    train_dataset = prepare_math_minst_data(train_ds, batch_size, subtask)
    test_dataset = prepare_math_minst_data(test_ds, batch_size, subtask)
    
    # Сheck the contents of the dataset
    for img1, img2, label in train_dataset:
        print(img1.shape, img2.shape, label.shape)
        break

    print('\n\n')

    # Instance model with Adam optimizer
    model = MyNNmodel(optimizer, subtask)

    # Pass arguments to training loop function
    train_loop(model=model,
                train_ds=train_dataset,
                test_ds=test_dataset,
                start_epoch=0,
                epochs=10,
                train_summary_writer=train_summary_writer,
                test_summary_writer=test_summary_writer,
                save_path=save_path)


In [7]:
def main():
    # Instance optimizers
    optimizer_Adam = tf.keras.optimizers.Adam(learning_rate=0.01)
    optimizer_SGD = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
    optimizer_SGD_no_momentum = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0) 
    optimizer_RMSprop = tf.keras.optimizers.RMSprop(learning_rate=0.01)
    optimizer_AdaGrad = tf.keras.optimizers.Adagrad(learning_rate=0.01)
    # Choose a path to save the weights
    save_path = "../Homework4/"

    subtask_number = np.array([1,2])
    batch_size = 32

    optimizers_list = [
        optimizer_Adam, 
        optimizer_SGD, 
        optimizer_SGD_no_momentum,
        optimizer_RMSprop,
        optimizer_AdaGrad
    ]

    for optimizer in optimizers_list:
        # Train model for the first subtask
        train(optimizer, save_path, batch_size, subtask_number[0])
        # Train model for the second subtask
        train(optimizer, save_path, batch_size, subtask_number[1])

    
if __name__ == '__main__':
    main()    


(32, 784) (32, 784) (32,)





100%|█████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:18<00:00, 103.75it/s]


Epoch: 0
['accuracy: 0.8417999744415283', 'loss: 2.4104807376861572']
['test_accuracy: 0.8388000130653381', 'test_loss: 2.4588167667388916']


100%|█████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:12<00:00, 153.31it/s]


Epoch: 1
['accuracy: 0.8427666425704956', 'loss: 2.3977105617523193']
['test_accuracy: 0.8385000228881836', 'test_loss: 2.4633843898773193']


...

Epoch: 7
['accuracy: 0.6888333559036255', 'loss: 0.13331729173660278']
['test_accuracy: 0.6859999895095825', 'test_loss: 0.13199804723262787']


100%|██████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:24<00:00, 76.16it/s]


Epoch: 8
['accuracy: 0.6964499950408936', 'loss: 0.1201329454779625']
['test_accuracy: 0.6823999881744385', 'test_loss: 0.12201423943042755']


100%|██████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:20<00:00, 89.98it/s]


Epoch: 9
['accuracy: 0.6996666789054871', 'loss: 0.11450948566198349']
['test_accuracy: 0.6894999742507935', 'test_loss: 0.11077160388231277']
